# **Exogenous Data Preprocessing Notebook**
## In this notebook we will preprocess our exogenous data, first creating our lag windows and the custom features that come from expanding the resulting view.  Then we will do any necessary cleaning such as reviewing outliers and missing values in our expanded data, imputing as needed so we have a clean dataset going into the next phase of the project.

#### Let's start by bringing in the libraries and logic necessary for reading in our file.

In [1]:

import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats



#### Now let's read in our data that we need for this notebook.

In [2]:
# Now let's access the main core_stock_data.csv file
csv_path = os.path.join(project_root, 'data', 'exogenous_data.csv')
exo_data = pd.read_csv(csv_path, parse_dates=['Date'], index_col= 'Date')
print(exo_data.head())
print(exo_data.shape)

            interest_rates_10yr         gold  copper    platinum  silver  \
Date                                                                       
2019-03-14                2.630  1293.400024  2.8885  825.599976  15.101   
2019-03-15                2.593  1301.800049  2.9035  830.299988  15.253   
2019-03-18                2.602  1300.300049  2.9035  832.599976  15.251   
2019-03-19                2.614  1305.000000  2.9195  851.200012  15.301   
2019-03-20                2.535  1300.500000  2.9155  858.200012  15.245   

            crude_oil  natural_gas    corn   wheat  volatility_index  \
Date                                                                   
2019-03-14  58.610001        2.855  361.50  448.25             13.50   
2019-03-15  58.520000        2.795  373.25  462.25             12.88   
2019-03-18  59.090000        2.850  371.50  456.75             13.10   
2019-03-19  59.029999        2.874  371.25  456.50             13.56   
2019-03-20  59.830002        2.820 

#### The first step I want to undertake is handling and imputing any missing features.  I want to do this first so that when we create our lag windows and resulting custom features the new values will be calculated properly, and it will be easier to address our resulting outliers.  We will look at the same process that we used for our core stock data, a combination of ffill, bfill and linear interpolation for max efficiency of our missing values.

In [3]:
def fill_missing_vals(df):
    df.ffill(inplace = True)
    df.bfill(inplace = True)
    df.interpolate(method = 'linear', inplace = True)
    
    return df

exo_data = exo_data.copy()

for col in exo_data.columns:
    if exo_data[col].isnull().any():
        exo_data.loc[:, col] = fill_missing_vals(exo_data[[col]])[col]



print(exo_data.isna().sum())

interest_rates_10yr               0
gold                              0
copper                            0
platinum                          0
silver                            0
crude_oil                         0
natural_gas                       0
corn                              0
wheat                             0
volatility_index                  0
exchange_rate_usd_eur             0
exchange_rate_usd_jpy             0
sp500                             0
nasdaq_100                        0
dow_jones_industrial_average      0
consumer_confidence_index         0
vanguard_total_world_stock_etf    0
us_treasury_bond_etf              0
dtype: int64


#### Looks great, now let's create the lag windows and custom features for our secondary stocks.

In [4]:
# We will need to create a temp variable since we are making so many custom features, it will otherwise create a PerformanceWarning since when we execute the cell on our main df its too much at once.
# By storing all the custom features to a temp variable and then concatenating them in a separate step it takes the pressure off in the execute phase.
new_temp_columns = pd.DataFrame(index = exo_data.index)

# Now let's create a variable to house our features we want to create lag features for.
features_to_lag = ['interest_rates_10yr', 'gold', 'copper', 'platinum', 'silver', 'crude_oil',
                    'natural_gas', 'corn', 'wheat', 'volatility_index', 'exchange_rate_usd_eur',
                    'exchange_rate_usd_jpy', 'sp500', 'nasdaq_100', 'dow_jones_industrial_average',
                    'consumer_confidence_index', 'vanguard_total_world_stock_etf', 'us_treasury_bond_etf']

# Now we apply the lag windows with a small 3-day window size.
for feature in features_to_lag:
    for lag in range(1,4):
        new_temp_columns[f'{feature}_Lag_{lag}'] = exo_data[feature].shift(lag)

# Great, now from these new lag-based features let's expand on it further by creating some custom features using our existing ones.
# Moving Averages
    new_temp_columns[f'{feature}_MA_7'] = exo_data[feature].rolling(window=7).mean()
    new_temp_columns[f'{feature}_MA_30'] = exo_data[feature].rolling(window=30).mean()
    # Volatility
    new_temp_columns[f'{feature}_Std_7'] = exo_data[feature].rolling(window=7).std()
    new_temp_columns[f'{feature}_Std_30'] = exo_data[feature].rolling(window=30).std()
    # Rate of Change (Momentum)
    new_temp_columns[f'{feature}_RoC'] = exo_data[feature].pct_change() * 100
    # Difference Features
    new_temp_columns[f'{feature}_Diff'] = exo_data[feature].diff()

# Now we can concatenate onto our original core_stock_data.
exo_data = pd.concat([exo_data, new_temp_columns], axis = 1)

print(exo_data.shape)
print(exo_data.head())

print(exo_data.isnull().sum())

(1415, 180)
            interest_rates_10yr         gold  copper    platinum  silver  \
Date                                                                       
2019-03-14                2.630  1293.400024  2.8885  825.599976  15.101   
2019-03-15                2.593  1301.800049  2.9035  830.299988  15.253   
2019-03-18                2.602  1300.300049  2.9035  832.599976  15.251   
2019-03-19                2.614  1305.000000  2.9195  851.200012  15.301   
2019-03-20                2.535  1300.500000  2.9155  858.200012  15.245   

            crude_oil  natural_gas    corn   wheat  volatility_index  ...  \
Date                                                                  ...   
2019-03-14  58.610001        2.855  361.50  448.25             13.50  ...   
2019-03-15  58.520000        2.795  373.25  462.25             12.88  ...   
2019-03-18  59.090000        2.850  371.50  456.75             13.10  ...   
2019-03-19  59.029999        2.874  371.25  456.50             13.56  

C:\Users\ryanm\AppData\Local\Temp\ipykernel_1748\3770416175.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_temp_columns[f'{feature}_Lag_{lag}'] = exo_data[feature].shift(lag)
C:\Users\ryanm\AppData\Local\Temp\ipykernel_1748\3770416175.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_temp_columns[f'{feature}_Lag_{lag}'] = exo_data[feature].shift(lag)
C:\Users\ryanm\AppData\Local\Temp\ipykernel_1748\3770416175.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

#### It looks like through the creation of all of our new features there are just a few missing values that slipped through.  Let's re-run our imputation logic to take care of them.  Also since I see 0s in our .head() printout, we will address those in our function below as leaving those will cause issues down the road with our outlier calculations.

In [5]:

def fill_missing_vals(df):
    # Ensure we're only applying the logic to numeric columns
    numeric_cols = df.select_dtypes(include=[np.number])
    
    # Replace any 0s or negative values with extremely small positive integers
    numeric_cols = numeric_cols.apply(lambda x: np.where(x <= 0, 1e-10, x))
    
    # Impute missing values with the following methodologies    
    numeric_cols.ffill(inplace=True)
    numeric_cols.bfill(inplace=True)
    numeric_cols.interpolate(method='linear', inplace=True)
    
    # Update the original dataframe with the modified numeric columns
    df[numeric_cols.columns] = numeric_cols
    
    return df

exo_data = exo_data.copy()

# Apply fill_missing_vals only to numeric columns with missing values
exo_data = fill_missing_vals(exo_data)

print("Missing values handled:")
print(exo_data.isna().sum())

# We will also put in a check for extremely large values as we are seeing warnings for those below
def replace_large_values(df, threshold=1e9):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Apply logic only to numeric columns
    numeric_cols = df.select_dtypes(include=[np.number])
    
    # Debug print statement to see what and how many values exceed our given threshold.
    print("Values exceeding the threshold before replacement:")
    for col in numeric_cols.columns:
        large_vals = df[col] > threshold
        if not large_vals.any():
            print(f"Column '{col}' has values larger than the threshold:")
            print(df[col][large_vals])
    
    # Replace large values with NaN
    df[numeric_cols.columns] = numeric_cols.apply(lambda x: np.where(x > threshold, np.nan, x))
    
    # Apply the fill_missing_vals logic to impute the modified numeric columns
    df = fill_missing_vals(numeric_cols)
    
    return df

threshold_value = 1e9
exo_stock_data = replace_large_values(exo_data, threshold=threshold_value)

# Calculate the max and min values only for numeric columns
numeric_cols = exo_data.select_dtypes(include = [np.number])

print("Max value after replacement", numeric_cols.max().max())
print("Min value after replacement:", numeric_cols.min().min())


Missing values handled:
interest_rates_10yr            0
gold                           0
copper                         0
platinum                       0
silver                         0
                              ..
us_treasury_bond_etf_MA_30     0
us_treasury_bond_etf_Std_7     0
us_treasury_bond_etf_Std_30    0
us_treasury_bond_etf_RoC       0
us_treasury_bond_etf_Diff      0
Length: 180, dtype: int64
Values exceeding the threshold before replacement:
Column 'interest_rates_10yr' has values larger than the threshold:
Series([], Name: interest_rates_10yr, dtype: float64)
Column 'gold' has values larger than the threshold:
Series([], Name: gold, dtype: float64)
Column 'copper' has values larger than the threshold:
Series([], Name: copper, dtype: float64)
Column 'platinum' has values larger than the threshold:
Series([], Name: platinum, dtype: float64)
Column 'silver' has values larger than the threshold:
Series([], Name: silver, dtype: float64)
Column 'crude_oil' has values large

#### Great, now let's look into viewing and handling outliers.  We will first see the appearance of outliers using z_score with a standard std of 3, and then looking at the column feature spread.

#### First though we will create a save point and a separate csv of our unscaled secondary stock data, as we will need this in an upcoming notebook.

In [6]:
exo_data.to_csv(os.path.join(project_root, 'data', 'exo_data_unscaled.csv'), index = True)

exo_data.head()

,interest_rates_10yr,gold,copper,platinum,silver,crude_oil,natural_gas,corn,wheat,volatility_index,...,vanguard_total_world_stock_etf_Diff,us_treasury_bond_etf_Lag_1,us_treasury_bond_etf_Lag_2,us_treasury_bond_etf_Lag_3,us_treasury_bond_etf_MA_7,us_treasury_bond_etf_MA_30,us_treasury_bond_etf_Std_7,us_treasury_bond_etf_Std_30,us_treasury_bond_etf_RoC,us_treasury_bond_etf_Diff
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-14,2.630,1293.400024,2.8885,825.599976,15.101,58.610001,2.855,361.50,448.25,13.50,...,5.699997e-01,24.870001,24.870001,24.870001,24.967143,25.044,0.097076,0.102876,2.010424e-01,4.999924e-02
2019-03-15,2.593,1301.800049,2.9035,830.299988,15.253,58.520000,2.795,373.25,462.25,12.88,...,5.699997e-01,24.870001,24.870001,24.870001,24.967143,25.044,0.097076,0.102876,2.010424e-01,4.999924e-02
2019-03-18,2.602,1300.300049,2.9035,832.599976,15.251,59.090000,2.850,371.50,456.75,13.10,...,3.300018e-01,24.920000,24.870001,24.870001,24.967143,25.044,0.097076,0.102876,1.000000e-10,1.000000e-10
2019-03-19,2.614,1305.000000,2.9195,851.200012,15.301,59.029999,2.874,371.25,456.50,13.56,...,1.000214e-02,24.910000,24.920000,24.870001,24.967143,25.044,0.097076,0.102876,1.000000e-10,1.000000e-10
2019-03-20,2.535,1300.500000,2.9155,858.200012,15.245,59.830002,2.820,371.50,464.75,13.91,...,1.000000e-10,24.900000,24.910000,24.920000,24.967143,25.044,0.097076,0.102876,4.417695e-01,1.100006e-01


In [7]:

numeric_cols = exo_data.select_dtypes(include = [np.number])
z_scores = np.abs(stats.zscore(numeric_cols))
threshold = 3 # Common threshold starting std modifier

outliers = (z_scores > threshold)

print(outliers.sum(axis = 0))


interest_rates_10yr             0
gold                            0
copper                          0
platinum                        8
silver                          0
                               ..
us_treasury_bond_etf_MA_30      0
us_treasury_bond_etf_Std_7     23
us_treasury_bond_etf_Std_30    20
us_treasury_bond_etf_RoC       31
us_treasury_bond_etf_Diff      28
Length: 180, dtype: int64


#### This is great, just using a standard zscore we are able to isolate the vast majority of potential outliers.  No further activity is needed here.

# For the last phase of our preprocessing notebook we can now move on to scaling our dataframe.

In [8]:
# Initiate the scaler and transform the data.
scaler = StandardScaler()
scaled_data = scaler.fit_transform(exo_data)
exo_scaled = pd.DataFrame(scaled_data, index = exo_data.index, columns = exo_data.columns)

print(exo_scaled.head())

            interest_rates_10yr      gold    copper  platinum    silver  \
Date                                                                      
2019-03-14             0.109020 -2.157252 -1.047962 -1.293853 -1.757015   
2019-03-15             0.080561 -2.122515 -1.026817 -1.248374 -1.719625   
2019-03-18             0.087483 -2.128718 -1.026817 -1.226118 -1.720117   
2019-03-19             0.096713 -2.109282 -1.004263 -1.046136 -1.707818   
2019-03-20             0.035949 -2.127891 -1.009902 -0.978401 -1.721593   

            crude_oil  natural_gas      corn     wheat  volatility_index  ...  \
Date                                                                      ...   
2019-03-14  -0.529975    -0.294268 -1.127038 -1.329873         -0.877685  ...   
2019-03-15  -0.534518    -0.327440 -1.039349 -1.239960         -0.953523  ...   
2019-03-18  -0.505745    -0.297032 -1.052409 -1.275283         -0.926613  ...   
2019-03-19  -0.508773    -0.283763 -1.054275 -1.276889         -0.870

#### This looks good, let's save it so we can use later on in future notebooks.

In [9]:
exo_scaled.to_csv(os.path.join(project_root, 'data', 'exo_data_preprocessed.csv'), index = True)


### In summary we have taken our raw secondary stock data that we generated from our generate_secondary_stocks.py script and have preprocessed it for further purposes.  We have created a 3-day lag window and our custom features, addressed any missing values and imputed accordingly, and looked at outliers a bit and making sure that we established normal distribution in at least a few of our features for good measure.  We ended this notebook by scaling our new dataframe using StandardScaler so that it is ready to go upon next use.